# Naive Bayes

One of the **simplest** yet effective algorithm that should be tried to solve the classification problem is Naive Bayes Algorithm. It’s a probabilistic modell which is based on the Bayes’ theorem which is an equation describing the **relationship of conditional probabilities of statistical quantities**.

The Naive Bayes algorithm has **hardly any hyperparameters** and is recommended to use first in the event of classification problems. If this does not give satisfactory results, then more complex algorithms should be used.

## Conditional Probability

The probability of $A$ **given we already know** that $B$ has occured, is defined as

$$
P(A|B) = \frac{P(A \cap B)}{P(B)}
$$

Only the _portion_ of $A$ that is contained in $B$ could occur, hence the original probability of $A \cap B$ must be recalculated (or **scaled**) to reflect the fact that the _new_ sample space is $B$.

In a slightly redundandt way, the conditional probability can also be written as

$$
P(A|B) = P(A \cap B\,|B)
$$

which makes it easier to see how we calculate the probability for an event $A$: Writing $P(A \cap B\,|B)$ makes it obvious that we want the probability that an event $A \cap B$ happens, **scaled by the knowledge we already have** about the event $B$:

$$
P({\color{orange}{A \cap B}}\,|{\color{purple}B}) = \frac{P({\color{orange}{A \cap B}})}{P({\color{purple}B})}
$$

## Bayes Theorem

See also [Bayes' Theorem with Lego](https://www.countbayesie.com/blog/2015/2/18/bayes-theorem-with-lego).

Bayes theorem describes the probability of an event, based on **prior knowledge** that might be related to the event. For example, if the risk of health problems is known to increase with age, Bayes theorem allows the risk to an individual of a known age to be assessed more accurately than simply assuming that the individual is typical of the population as a whole.

$$
\begin{aligned}
P(A|B) &= \frac{P(B|A) \cdot P(A)}{P(B)} \\[10pt]
\text{Posterior} &= \, \frac{\text{Likelihood} \cdot \text{Prior}}{\text{Evidence}}
\end{aligned}
$$

with

* the conditional probability $P(A|B)$ of event $A$ occurring given that $B$ is true. This is also called **posterior probability**.
* the conditional probability $P(B|A)$ of event $B$ occurring given that $A$ is true. This is also called the **likelyhood**.
* the probability $P(A)$. This is also called the **prior probability**.
* the probability $P(B)$. This is also called the **evidence** which **normalizes** our probabilities.

If we are only interested in **proportions** of conditional probabilities, we can also write

$$
\begin{aligned}
P(A|B) &\propto P(B|A) \cdot P(A) \\[10pt]
\text{Posterior} &\propto \, \text{Likelihood} \cdot \text{Prior}
\end{aligned}
$$

### Alternative Form

Another form of Bayes theorem for **two competing statements** or hypotheses is

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B|A) \cdot P(A) + P(B|\neg A) \cdot P(\neg A)}
$$

For proposition $A$ and evidence or background $B$,

* $P(A)$ is the prior probability, the initial degree of belief in $A$.
* $P(\neg A)$ is the corresponding initial degree of belief in not $A$, that $A$ is false, where $P(\neg A) = 1 - P(A)$
* $P(B|A)$ is the conditional probability or likelihood, the degree of belief in $B$ given that proposition $A$ is true.
* $P(B|\neg A)$ is the conditional probability or likelihood, the degree of belief in $B$ given that proposition $A$ is false.
* $P(A|B)$ is the posterior probability, the probability of $A$ after taking into account $B$.

### Example

#### Problem

Knowing a medical test having a **99% accuracy** (for true positives and true negatives). Already knowing that **1 out of 10000 people are sick**, what is the probability of an individual being sick, given that this individual got a positive test result?

#### Solution

![false-positives](images/bayes-theorem.svg)

What we knew before we knew the test is positive, is the **prior probability** $P(sick) = 0.0001$ and $P(healthy) = 0.9999$. 

As only the **positive tests** actually occured, we scale the likelyhood and the prior with the **evidence** $P(positive)$:

$$
\begin{align}
P(positive) &= P(sick) \cdot P(positive|sick) + P(healthy) \cdot P(positive|healthy) \\[10pt]
&= P(sick) \cdot \text{Sensitivity} + (1 - P(sick)) \cdot (1 - \text{Specificity})
\end{align}
$$

In [15]:
# P(sick)
p_sick = 0.0001

# P(~sick) or P(healthy)
p_healthy = 1 - p_sick

# Sensitivity or P(positive|sick)
p_positive_sick = 0.99

# Specificity or P(positive|healthy)
p_positive_healthy = 1 - p_positive_sick

# P(positive)
p_positive = (p_sick * p_positive_sick) + (p_healthy * p_positive_healthy)

print(f'The probability of getting a positive test result P(positive) is: {p_positive:.4f}')

The probability of getting a positive test result P(positive) is: 0.0101


The **posterior probability**, what we infered after we knew that the test is positive, is:

$$
\begin{align}
P(sick|positive) &= \frac{P(sick) \cdot P(positive|sick)}{P(positive)} \\[10pt]
&= \frac{P(sick) \cdot P(positive|sick)}{P(sick) \cdot P(positive|sick) + P(healthy) \cdot P(positive|healthy)} \\[10pt]
&= \frac{0.0001 \cdot 0.99}{0.0001 \cdot 0.99 + 0.9999 \cdot 0.01} \\[10pt]
&= 0.0098 \approx 1 \%
\end{align}
$$

In [19]:
p_sick_positive = p_sick * p_positive_sick / p_positive
print(f'The posterior probability of being sick having a positive test result is: {p_sick_positive:.4f}')

p_healthy_positive = p_healthy * p_positive_healthy / p_positive
print(f'The posterior probability of being healthy having a positive test result is: {p_healthy_positive:.4f}')

The posterior probability of being sick having a positive test result is: 0.0098
The posterior probability of being healthy having a positive test result is: 0.9902


The sum of our posteriors will always equal `1`.

In [20]:
print(f'{(p_sick_positive + p_healthy_positive):.4f}')

1.0000


## Naive Bayes

Lets consider a spam classifier with event $A$ **being spam** and event $B$ **containing the words** $w_1, \dots, w_n$, Naive Bayes considers only proportions:

$$
\begin{align}
P(A|B) &\propto P(B|A) \cdot P(A) \\[6pt]
P(spam \, | \, w_1, \dots, w_n) &\propto P(w_1, \dots, w_n \, | \, spam) \cdot P(spam)
\end{align}
$$

Naive Bayes does now mathematically wrongly calculate

$$
\begin{align}
P(w_1, \dots, w_n \, | \, spam) &= P(w_1 \cap w_2 \cap \ldots \cap w_n \, | \, spam) \\
&= P(w_1|spam) \cdot P(w_2|spam) \cdot \ldots \cdot P(w_n|spam)
\end{align}
$$

The probability that a word $w_i$ occurs in a spam message is the number $n_i$ of the words $w_i$ in all spam messages related to the total number of words $N$ of all spam messages. We add a value $\alpha$ to avoid having a probability of $0$ for a word:

$$
P(w_i|spam) = \frac{n_i}{N} + \alpha
$$

Typically, we set $\alpha = 1$.

Naive Bayes is considered **naive**, because it treats all words of a language as a **bag of words** regardless of the order or context of the words. Ignoring relationships among words has a **high bias**, but because it works well in practice, it has a **low variance**.

So, the **naive** bit of the theorem is when it considers each **feature** to be **independent** of each other which may not always be the case.

### Example

The initial guess that we observe a spam message $P(spam)$ is called a **prior probability**. This guess can be any probability we want, but a common guess is estimated from the training data.

Now we multiply the initial guess with the probabilities that the words $w_i$ occur in a normal message: $P(spam) \cdot P(w_i|spam) \cdot \ldots \cdot P(w_j|spam)$. These probabilities are derived from test-data.

This is proportional to the **probability that a message is spam** given the words $w_1, \dots, w_n$, hence can be considered as a **score**.

If we do the same for non-spam messages, we get two scores which tell us whether a message is spam or not.



## Naive Bayes in SciKit-Learn

In [4]:
from sklearn.naive_bayes import GaussianNB           # Gaussian Naive Bayes:    continuous value and are not discrete
from sklearn.naive_bayes import BernoulliNB          # Bernoulli Naive Bayes:   Multinomial Naive Bayes for boolean classes (e.g. spam/ham)
from sklearn.naive_bayes import MultinomialNB        # Multinomial Naive Bayes: probability of observing counts among a number of categories

### Naive Bayes Spam Classifier